### Setting Envirionment

In [1]:
# ! pip install pandas
# ! pip install openai
# ! pip install langchain

In [2]:
import os
import json
import openai
import pandas as pd
import csv
import random
import string
import datetime

In [3]:
with open('openai_api_key.txt', 'r') as f:
    openai.api_key = f.read().strip()
os.environ["OPENAI_API_KEY"] = openai.api_key
#os.environ.get("OPENAI_API_KEY") # api_key check if needed

In [4]:
# Calling prompts
with open('sprompt_int.txt', 'r') as f:
    sprompt_int = f.read()

with open('sprompt_obj.txt', 'r') as f:
    sprompt_obj = f.read()

with open('sprompt_plt.txt', 'r') as f:
    sprompt_plt = f.read()

In [5]:
# # Checking the prompts
# print(sprompt_int[-30:-1])
# print(sprompt_obj[-30:-1])
# print(sprompt_plt[-30:-1])

### Scene Genaration with Langchain Schema

In [6]:
# Calling Lanchain functions
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

- [cls0] OBJECT UNCERTAINTY

In [7]:
# OBJECT UNCERTAINTY 
chat = ChatOpenAI(model="gpt-4-0314", temperature=1)

messages = [
    SystemMessage(content=f"'{sprompt_obj}'"),
    HumanMessage(content="Write sentences each about a situation with an ordinary object that is out of context."),
]

scene = chat(messages)
split_output_lists = [sentence.strip() for sentence in scene.content.split(';')]

# Create a dataframe with a scene class
df0 = pd.DataFrame({
    'scn_sentence': split_output_lists,
    'scn_cls': [0] * len(split_output_lists),
    'model' : chat.model_name,
    'temp' : chat.temperature
})

- [cls1] INTENTION UNCERTAINTY

In [8]:
# INTENTION UNCERTAINTY
chat = ChatOpenAI(model="gpt-4-0314", temperature=1)

messages = [
    SystemMessage(content=f"'{sprompt_int}'"),
    HumanMessage(content="Write short sentences each of which is about a realistic situation that gives a question mark about a character's behavior."),
    ]

scene = chat(messages)
split_output_lists = [sentence.strip() for sentence in scene.content.split(';')]

# Create a dataframe with a scene class
df1 = pd.DataFrame({
    'scn_sentence': split_output_lists,
    'scn_cls': [1] * len(split_output_lists),
    'model' : chat.model_name,
    'temp' : chat.temperature
})

- cls[2] EVENT UNCERTAINTY

In [9]:
# EVENT UNCERTAINTY
chat = ChatOpenAI(model="gpt-4", temperature=1)

messages = [
    SystemMessage(content=f"'{sprompt_plt}'"),
    HumanMessage(content="Write sentences that depict a noticeable change in a ordinary scenario involving a person or object."),
    ]

scene = chat(messages)
split_output_lists = [sentence.strip() for sentence in scene.content.split(';')]

# Create a dataframe with a scene class
df2 = pd.DataFrame({
    'scn_sentence': split_output_lists,
    'scn_cls': [2] * len(split_output_lists),
    'model' : chat.model_name,
    'temp' : chat.temperature
})

In [10]:
# Row-wise concatenating dataframes
df = pd.concat([df0, df1, df2], axis=0)

In [11]:
# Row-wise concatenating dataframes
df = pd.concat([df0, df1, df2], axis=0)

### Wirte the final DF with random IDs

In [12]:
# Generate a unique random 6-digit ID using a mix of 3 numbers and 3 alphabets
def generate_random_id():
    numbers = ''.join(random.sample(string.digits, 3))  # picking 3 numbers
    letters = ''.join(random.sample(string.ascii_uppercase, 3))  # picking 3 uppercase alphabets
    random_id = ''.join(random.sample(numbers + letters, 6)) # Combining and shuffling the numbers and alphabets
    return random_id 

# Generate a non-overlapping random IDs
unique_ids = set()
while len(unique_ids) < len(df):
    unique_id = generate_random_id()
    if unique_id not in unique_ids:
        unique_ids.add(unique_id)

# add the IDs for each sentence by inserting a new column at the leftmost part of the df
df.insert(0, 'scn_id', list(unique_ids))

In [13]:
# # Sort in ascending order by 'scn_id' (optional)
# df = df.sort_values(by='scn_id')

### Saving files to CSV

In [14]:
# getting the current time
now = datetime.datetime.now()
# Convert to YYMMDD_HHMMSS format
FileName = now.strftime("%y%m%d_%H%M%S")
print(FileName)

231013_143611


In [15]:
# Save DataFrame to CSV
df.to_csv(f'_output_scene/scene_{FileName}.csv', index=False)